In [14]:
import pandas as pd
import FinanceDataReader as fdr
from tqdm import tqdm

## Preprocess

In [40]:
NO_KOS_CODES = ['003340']  # 코스피, 코스닥 종목이 아닌 것

In [2]:
def split_reporter(s):
    s_splited = s.split()
    if len(s_splited) == 1:
        return s_splited[0], 'None'
    elif len(s_splited) == 2:
        return s_splited[0], s_splited[1]
    else:
        return s_splited[0], ' '.join(s_splited[1:])

In [45]:
df = pd.read_csv('data/report_230101_230531.csv')

In [46]:
df['codes'] = df.apply(lambda x:x['titles'].split(' A')[1].split('-')[0].strip(), axis=1)

In [6]:
# df['report_company'] = df.apply(lambda x:split_reporter(x['reporter'])[0], axis=1)
# df['report_name'] = df.apply(lambda x:split_reporter(x['reporter'])[1], axis=1)
# df = df.drop(columns=['titles', 'reporter'])

In [47]:
df = df.drop(columns=['titles'])

In [48]:
df = df[~df['codes'].isin(NO_KOS_CODES)]

In [8]:
# df = df.drop_duplicates(subset=['price_goal', 'reporter'], keep='first').reset_index(drop=True)

In [49]:
df = df.drop_duplicates(keep='first').reset_index(drop=True)

In [145]:
df['price_goal'] = df['price_goal'].str.replace(',', '').astype(float)

### EDA

In [146]:
data_list = []
for row in tqdm(range(len(df))):
    code, date = df.loc[row, 'codes'], df.loc[row, 'dates']
    date = date.replace('/', '-')
    df_stock = pd.read_csv(f'data_stock/{code}.csv')
    df_stock_target = df_stock[df_stock['Date'] >=date].reset_index(drop=True)
    max_n = len(df_stock_target)
    data = {}
    for n in [0, 1, 5, 10, 20]:
        if n == 1:
            data[f'{n}_Open'] = df_stock_target.loc[n]['Open']
        if n < max_n:
            data[f'{n}_Close'] = df_stock_target.loc[n]['Close']
        else:
            data[f'{n}_Close'] = -1
    data_list.append(data)

100%|██████████| 10825/10825 [00:15<00:00, 676.67it/s]


In [147]:
df_ = pd.DataFrame(data_list)

In [149]:
df_concat = pd.concat([df, df_], axis=1)

In [153]:
df_concat['price_goal_ratio'] = df_concat['price_goal'] / df_concat['0_Close'] - 1

In [197]:
n = 10

In [198]:
df_target = df_concat[df_concat[f'1_Open'] != 0].reset_index(drop=True)
df_target = df_target[df_target[f'{n}_Close'] != -1].reset_index(drop=True)

In [199]:
df_target['ratio'] = df_target[f'{n}_Close'] / df_target['1_Open'] - 1

In [200]:
df_target['ratio'].mean()

0.006798407872241865

In [201]:
df_target[df_target['price_goal_ratio'] > 0.5]['ratio'].mean()

0.014793360064230534

In [202]:
len(df_target)

10239

In [203]:
sum(df_target['price_goal_ratio'] > 0.5)

1233

In [168]:
df_mean = df_target.groupby('reporter').mean()['ratio']
df_count = df_target.groupby('reporter').count()['ratio']

In [169]:
df_result = pd.concat([df_count, df_mean], axis=1)

In [170]:
df_result.columns = ['count', 'mean']

In [131]:
df_result.sort_values('mean').tail(20)

,count,mean
reporter,,
NH투자증권 심의섭,12,0.102649
키움증권 권준수,18,0.105135
이베스트투자증권 조은애,19,0.105798
현대차증권 곽민정,34,0.106594
삼성증권 정동희,8,0.107374
SK증권 윤혁진,3,0.114275
부국증권 김성환,5,0.122805
삼성증권 장정훈,46,0.127844
IBK투자증권 이건재,10,0.129200


In [114]:
df_target['ratio'].mean()

0.0123737337425227

In [204]:
df

,invest_opinion,price_goal,reporter,contents,dates,codes
0,BUY,90000.0,DB금융투자 김평모,"4Q22, 컨센서스 하회 전망\n- 23년 글로벌 자동차 수요 부진",2023/01/02,000270
1,매수,100000.0,한국투자증권 김진우,4분기 영업이익 컨센서스 10% 상회 전망\n- 실적 고점보다 저점이 궁금한 시기,2023/01/02,000270
2,BUY,85000.0,교보증권 남주신,전기차 SUV 시리즈에 주목\n- EV9으로 커진 밸류에이션 매력,2023/01/02,000270
3,BUY,47000.0,유안타증권 이현수,"4Q22 OP estimated at W58bn (-92% YoY, -84% QoQ...",2023/01/02,004020
4,BUY,47000.0,유안타증권 이현수,"4Q22(E): 영업이익 580억원 추정 (yoy -92%, qoq -84%)\n-...",2023/01/02,004020
...,...,...,...,...,...,...
10820,매수,9000.0,한국투자증권 강경태,"리파이낸싱 첫 해, 12기부터 경상 배당금 감소할 것\n- 자산 매각 차익으로 지급...",2023/05/31,293940
10821,매수,5500.0,한국투자증권 강경태,"리파이낸싱 첫 해, DPS 감소할 것\n- 자산 편입 활동 지속, 이천7물류센터 신...",2023/05/31,365550
10822,매수,100000.0,IBK투자증권 김태현,"2분기 실적, 낮아진 시장 기대치에 부합할 전망\n- 건기식 부문이 담배 및 부동산...",2023/05/31,033780
10823,매수,2600.0,한국투자증권 최고운,2분기 벌크해운 운임은 경기부진 우려로 약세가 지속\n- 단기 불확실성 확대를 감안...,2023/05/31,005880


### workspace

In [ ]:
codes = df['codes'].unique()

In [ ]:
## fdr에서 데이터 local로 다운로드
# for code in tqdm(codes):
#     df_stock = fdr.DataReader(code, '2023')
#     df_stock.to_csv(f'data_stock/{code}.csv')